# How to build a new universe?

## Data preprocessing
This is a jupyter version of CLI tutorial that can be found [here](). You will use here python functions insted of CLI to build and assess diffrent universe. Fielse that you will use here can be downlodead from XXX. In there you will find a compressed folder:

```
consensus:
    - raw
        test_1.bed
        test_2.bed
        test_3.bed
        test_4.bed
    file_list.txt
    chrom.sizes
```

In the raw folder there are example BED files used in this tutorial and file withe names of files we will analyze.

It assummes that you alread have files of the genome coverage by the analzed colletion. The example of how to creat them can be found [here]().

## Coverage cutoff universe

First, you will create a coverage cutoff universe (CC). This is the simplest type of a universe that only includes genomic positions with coverage greater or equal to cutoff *x*. This cutoff by default is calculated using simple likelihood model that calculates the probability of appearing in a collection. The universe can be build just based on genome coverage:

In [1]:
from geniml.universe.cc_universe import cc_universe
cc_universe("coverage/", file_out="universe_cc_py.bed")

Depending on the task the universe can be smooth by setting ```merge``` option with the distance beloved witch peaks should be merged together and 
`filter_size` with minimum size of peak that should be part of the universe. Instead of it using maximum likelihood cutoff one can also defined cutoff with `cutoff` option. If it is set to 1 the result is union universe, and when to number of files it wil produce intersection universe:

In [2]:
cc_universe("coverage/", file_out="universe_union.bed", cutoff=1)
cc_universe("coverage/", file_out="universe_intersection.bed", cutoff=4)

## Coverage cutoff flexible universe
A more complex version of coverage cutoff universe is coverage cutoff flexible universe (CCF). In contrast to its' fixed version it produces flexible universes. It uses two cutoffs calculated based on maximum likelihood cutoff, making a confidence interval around the optimal cutoff value. Despite the fact that the CFF universe is more complex it is build using the same input as the CC universe: 


In [3]:
from geniml.universe.ccf_universe import ccf_universe

ccf_universe("coverage/", file_out="universe_ccf_py.bed")

## Maximum likelihood universe
In the previous examples both CC anf CCF universes used simple likelihood model to calculate the cutoff. However, we also developed more complex likelihood model that takes into account the positions of starts and ends of the regions in the collection. This LH model can build based on coverage files:

In [4]:
from geniml.likelihood.build_model import main

main("model_py.tar", "coverage/",
     "all",
     file_no=4)

Function 'main' executed in 0.0007min


The resulting tar archiver contains LH model that can be used for building flexible universes called a maximum likelihood universe (ML):

In [5]:
from geniml.universe.ml_universe import ml_universe

ml_universe("model_py.tar",
     "coverage",
     "all",
     "universe_ml_py.bed")

numba is installed


## HMM 
The forth presented method of creating universes utilizes Hidden Markov Models. In this approach the parts of flexible regions are hidden states of the model, while genome coverage by the collections are emissions. The resulting universe is called Hidden Markov Model universe. It can be build only based on the genome coverage by the collection:

In [6]:
from geniml.universe.hmm_universe import hmm_universe

hmm_universe("coverage/",
             "universe_hmm_py.bed")

# How to assess new universe?

So far you used many different methods for creating new universes. But choosing, which universe represents data the best can be challenging. To help with this decision we created three different metrics for assessing universe fit to the region collections: a base-level overlap score (F10), a region boundary distance score (RBD), and a likelihood score (LH). Here we present an example, which calculates all metrics for HMM universe:

In [7]:
from geniml.assess.assess import get_f_10_score

f10 = get_f_10_score(
    "raw/",
    'file_list.txt',
    "universe_hmm_py.bed",
    1)

f"Universe F10: {f10:.2f}"

'Universe F10: 0.93'

In [8]:
from geniml.assess.assess import get_mean_rbs
rbs = get_mean_rbs("raw/",
    'file_list.txt',
    "universe_hmm_py.bed", 1)
f"Universe RBS: {rbs:.2f}"

'Universe RBS: 0.77'

In [9]:
from geniml.assess.assess import get_likelihood
lh = get_likelihood(
    "model_py.tar",
    "universe_hmm_py.bed",
    "coverage/"
)
f"Universe LH: {lh:.2f}" 

'Universe LH: -127156.87'

Both region baounary score and likelihood can cacluated taking into account universe flexiblility:

In [10]:
from geniml.assess.assess import get_mean_rbs
rbs_flex = get_mean_rbs(
    "raw/",
    'file_list.txt',
    "universe_hmm_py.bed",
    1,
    flexible=True)
f"Universe flexible RBS: {rbs_flex:.2f}"

'Universe flexible RBS: 0.98'

In [11]:
lh_flex = get_likelihood(
    "model_py.tar",
    "universe_hmm_py.bed",
    "coverage/"
)
f"Universe flexible LH: {lh_flex:.2f}" 

'Universe flexible LH: -127156.87'

Additionaly using `get_rbs_from_assessment_file` and `get_f_10_score_from_assessment_file` you can calculate the assessment metrics from prevously calaculated assessment file.`